__NOTE:__ The networks are not available in this github page. This notebook is provided to make it easier to see the results ran in the thesis.

# WBC

In [8]:
import copy
import numpy as np
import torch
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import MinMaxScaler
from config import config
import sys
import plot_functions as pf
import pipeline_functions as pip_func
sys.path.append('networks')
from lrt_net import BayesianNetwork
import torch.nn.functional as F

In [10]:
# importlib.reload(pf)
for i in range(10):
    net = torch.load(f"trained_nets/wbc/flow/net{i}", map_location=torch.device('cpu'))
    pf.save_metrics(net, path=f"results/wbc/flow/net{i}")
    pf.run_path_graph(net, threshold=0.5, save_path=f"path_graphs/wbc/flow/net{i}", show=False)

{'Prob I0 from I': 0, 'Prob I1 from I': 0, 'Prob I2 from I': 0, 'Prob I3 from I': 0, 'Prob I4 from I': 0, 'Prob I5 from I': 0, 'Prob I6 from I': 0, 'Prob I7 from I': 0, 'Prob I8 from I': 0, 'Prob I9 from I': 0, 'Prob I10 from I': 0, 'Prob I11 from I': 0, 'Prob I12 from I': 0, 'Prob I13 from I': 0, 'Prob I14 from I': 0, 'Prob I15 from I': 0, 'Prob I16 from I': 0, 'Prob I17 from I': 0, 'Prob I18 from I': 0, 'Prob I19 from I': 0, 'Prob I20 from I': 0, 'Prob I21 from I': 0, 'Prob I22 from I': 0, 'Prob I23 from I': 0, 'Prob I24 from I': 0, 'Prob I25 from I': 0, 'Prob I26 from I': 0, 'Prob I27 from I': 0, 'Prob I28 from I': 0, 'Prob I29 from I': 0, 'Prob I0 from H1': 0, 'Prob I1 from H1': 0, 'Prob I2 from H1': 0, 'Prob I3 from H1': 0, 'Prob I4 from H1': 0, 'Prob I5 from H1': 0, 'Prob I6 from H1': 0, 'Prob I7 from H1': 0, 'Prob I8 from H1': 0, 'Prob I9 from H1': 0, 'Prob I10 from H1': 0, 'Prob I11 from H1': 0, 'Prob I12 from H1': 0, 'Prob I13 from H1': 0, 'Prob I14 from H1': 0, 'Prob I15 from

In [11]:
import numpy as np

res = {}
data = np.loadtxt(f"results/flow_class_skip_2_hidden_50_dim_20000_epochs_0.005_lr_wbc_full.txt", delimiter=",")
data_median = np.loadtxt(f"results/flow_class_skip_2_hidden_50_dim_20000_epochs_0.005_lr_wbc_median.txt", delimiter=",")
res["median_acc_full"] = np.median(data, 0)[0]
res["min_acc_full"] = np.min(data, 0)[0]
res["max_acc_full"] = np.max(data, 0)[0]

res["median_acc_median"] = np.median(data_median, 0)[0]
res["min_acc_median"] = np.min(data_median, 0)[0]
res["max_acc_median"] = np.max(data_median, 0)[0]

res["median_density"] = np.median(data, 0)[1]
res["min_density"] = np.min(data, 0)[1]
res["max_density"] = np.max(data, 0)[1]

res["median_used_weights"] = np.median(data_median, 0)[1]
res["min_used_weights"] = np.min(data_median, 0)[1]
res["max_used_weights"] = np.max(data_median, 0)[1]

res

{'median_acc_full': 0.9026315789473685,
 'min_acc_full': 0.6947368421052631,
 'max_acc_full': 0.9263157894736842,
 'median_acc_median': 0.9473684210526315,
 'min_acc_median': 0.9087719298245613,
 'max_acc_median': 0.963157894736842,
 'median_density': 0.0014336917083710432,
 'min_density': 0.0008960572886280715,
 'max_density': 0.001792114577256143,
 'median_used_weights': 8.0,
 'min_used_weights': 5.0,
 'max_used_weights': 10.0}

In [12]:
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats

import os
import sys
current_dir = os.getcwd()
sys.path.append('layers')
from config import config

# Parameters from config file
HIDDEN_LAYERS = config['n_layers'] - 2 
epochs = config['num_epochs']
dim = config['hidden_dim']
n_nets = config['n_nets']

path = "results/wbc/flow/"
res = {}


res["used_weights"] = []

for n in range(n_nets):
    median_info = np.load(path+f"net{n}_median.npy", allow_pickle=True).item()
    res["used_weights"].append(median_info["used_weights"])

print("USED WEIGHTS:")
m = (res["used_weights"])
print(f"median: {np.median(m)}")
print(f"min: {np.min(m)}")
print(f"max: {np.max(m)}\n")

USED WEIGHTS:
median: 8.0
min: 5
max: 10



In [13]:
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats

import os
import sys
current_dir = os.getcwd()
sys.path.append('layers')
from config import config

# Parameters from config file
HIDDEN_LAYERS = config['n_layers'] - 2 
epochs = config['num_epochs']
dim = config['hidden_dim']
n_nets = config['n_nets']

path = "results/wbc/flow/"
res = {}
res["median_depth"] = []
res["max_width"] = []

for n in range(n_nets):
    median_info = np.load(path+f"net{n}_median.npy", allow_pickle=True).item()
    res["median_depth"].append(median_info["avg_path_length"])
    search = True
    i = 0
    while search:
        search = np.sum(median_info["include_inputs"][i]) == 0
        i += 1

    res["max_width"].append(HIDDEN_LAYERS-i+2)

print("Avg. depth")
print(np.median(res["median_depth"]))
print(np.min(res["median_depth"]))
print(np.max(res["median_depth"]))
print("")

print("Max depth")
print(np.median(res["max_width"]))
print(np.min(res["max_width"]))
print(np.max(res["max_width"]))
print("")

Avg. depth
1.1339285714285714
1.0
1.4285714285714286

Max depth
2.0
1
2



In [8]:
import numpy as np


import os
import sys
current_dir = os.getcwd()
sys.path.append('layers')
from config import config

# Parameters from config file
HIDDEN_LAYERS = config['n_layers'] - 2 
epochs = config['num_epochs']
dim = config['hidden_dim']
n_nets = config['n_nets']

path = "results/wbc/flow/"
res = {}
for i in range(p):
    res[f"x{i+1}_inclution_rate"] = 0
    
    for n in range(n_nets):
        median_info = np.load(path+f"net{n}_median.npy", allow_pickle=True).item()
        #res[d][f"x{i+1}_inclution_rate"] += (median_info["expected_depth_input"][i]>0)
        count = 0
        for j in median_info["include_inputs"]:
            count += j[i]

        res[f"x{i+1}_inclution_rate"] += (count>0)
    res[f"x{i+1}_inclution_rate"] /= n_nets

res

{'x1_inclution_rate': 0.6,
 'x2_inclution_rate': 0.0,
 'x3_inclution_rate': 0.2,
 'x4_inclution_rate': 0.0,
 'x5_inclution_rate': 0.0,
 'x6_inclution_rate': 0.0,
 'x7_inclution_rate': 0.0,
 'x8_inclution_rate': 1.0,
 'x9_inclution_rate': 0.0,
 'x10_inclution_rate': 0.0,
 'x11_inclution_rate': 0.1,
 'x12_inclution_rate': 0.0,
 'x13_inclution_rate': 0.0,
 'x14_inclution_rate': 0.0,
 'x15_inclution_rate': 0.0,
 'x16_inclution_rate': 0.0,
 'x17_inclution_rate': 0.0,
 'x18_inclution_rate': 0.0,
 'x19_inclution_rate': 0.0,
 'x20_inclution_rate': 0.0,
 'x21_inclution_rate': 0.8,
 'x22_inclution_rate': 1.0,
 'x23_inclution_rate': 0.9,
 'x24_inclution_rate': 0.4,
 'x25_inclution_rate': 0.0,
 'x26_inclution_rate': 0.0,
 'x27_inclution_rate': 0.0,
 'x28_inclution_rate': 1.0,
 'x29_inclution_rate': 0.0,
 'x30_inclution_rate': 0.0}

In [4]:
load_breast_cancer().feature_names[[0,1,2,6,7,20,21,22,23,26,27]]

array(['mean radius', 'mean texture', 'mean perimeter', 'mean concavity',
       'mean concave points', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst concavity',
       'worst concave points'], dtype='<U23')

# Abalone

In [2]:
import copy
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
from sklearn.model_selection import train_test_split
from config import config
import sys
import plot_functions as pf
import pipeline_functions as pip_func
sys.path.append('networks')
from flow_net import BayesianNetwork
import torch.nn.functional as F

CPUs are used!


In [5]:
# importlib.reload(pf)
for i in range(10):
    net = torch.load(f"trained_nets/abalone/flow/net{i}", map_location=torch.device('cpu'))
    pf.save_metrics(net, path=f"results/abalone/flow/net{i}")
    pf.run_path_graph(net, threshold=0.5, save_path=f"path_graphs/abalone/flow/net{i}", show=False)

{'Prob I0 from I': 0, 'Prob I1 from I': 0, 'Prob I2 from I': 0, 'Prob I3 from I': 0, 'Prob I4 from I': 0, 'Prob I5 from I': 0, 'Prob I6 from I': 0, 'Prob I7 from I': 0, 'Prob I8 from I': 0, 'Prob I0 from H1': 0, 'Prob I1 from H1': 0, 'Prob I2 from H1': 0, 'Prob I3 from H1': 0, 'Prob I4 from H1': 0, 'Prob I5 from H1': 0, 'Prob I6 from H1': 0, 'Prob I7 from H1': 0, 'Prob I8 from H1': 0, 'Prob I0 from H2': 0, 'Prob I1 from H2': 0, 'Prob I2 from H2': 0, 'Prob I3 from H2': 0, 'Prob I4 from H2': 0, 'Prob I5 from H2': 0, 'Prob I6 from H2': 0, 'Prob I7 from H2': 0, 'Prob I8 from H2': 0}
H2
{'Prob I0 from I': 0, 'Prob I1 from I': 0, 'Prob I2 from I': 0, 'Prob I3 from I': 0, 'Prob I4 from I': 0, 'Prob I5 from I': 0, 'Prob I6 from I': 0, 'Prob I7 from I': 0, 'Prob I8 from I': 0, 'Prob I0 from H1': 0, 'Prob I1 from H1': 0, 'Prob I2 from H1': 0, 'Prob I3 from H1': 0, 'Prob I4 from H1': 0, 'Prob I5 from H1': 0, 'Prob I6 from H1': 0, 'Prob I7 from H1': 0, 'Prob I8 from H1': 0, 'Prob I0 from H2': 0, '

In [23]:
import numpy as np

res = {}
data = np.loadtxt(f"results/flow_reg_skip_2_hidden_200_dim_3000_epochs_0.005_lr_abalone_full.txt", delimiter=",")
data_median = np.loadtxt(f"results/flow_reg_skip_2_hidden_200_dim_3000_epochs_0.005_lr_abalone_median.txt", delimiter=",")
res["median_rmse_full"] = np.median(data, 0)[0]
res["min_rmse_full"] = np.min(data, 0)[0]
res["max_rmse_full"] = np.max(data, 0)[0]

res["median_rmse_median"] = np.median(data_median, 0)[0]
res["min_rmse_median"] = np.min(data_median, 0)[0]
res["max_rmse_median"] = np.max(data_median, 0)[0]

res["median_density"] = np.median(data, 0)[1]
res["min_density"] = np.min(data, 0)[1]
res["max_density"] = np.max(data, 0)[1]

res["median_used_weights"] = np.median(data_median, 0)[1]
res["min_used_weights"] = np.min(data_median, 0)[1]
res["max_used_weights"] = np.max(data_median, 0)[1]

res["median_r2_full"] = np.median(data, 0)[2]
res["min_r2_full"] = np.min(data, 0)[2]
res["max_r2_full"] = np.max(data, 0)[2]

res["median_r2_median"] = np.median(data_median, 0)[2]
res["min_r2_median"] = np.min(data_median, 0)[2]
res["max_r2_median"] = np.max(data_median, 0)[2]

res["median_corr_full"] = np.median(np.sqrt(data), 0)[2]
res["min_corr_full"] = np.min(np.sqrt(data), 0)[2]
res["max_corr_full"] = np.max(np.sqrt(data), 0)[2]

res["median_corr_median"] = np.median(np.sqrt(data_median), 0)[2]
res["min_corr_median"] = np.min(np.sqrt(data_median), 0)[2]
res["max_corr_median"] = np.max(np.sqrt(data_median), 0)[2]

res

{'median_rmse_full': 2.1400747299194336,
 'min_rmse_full': 2.1222639083862305,
 'max_rmse_full': 2.182795763015747,
 'median_rmse_median': 2.114986300468445,
 'min_rmse_median': 2.1052451133728027,
 'max_rmse_median': 2.186708450317383,
 'median_density': 0.0005934853106737137,
 'min_density': 0.00034239538945257664,
 'max_density': 0.0008217489230446517,
 'median_used_weights': 26.0,
 'min_used_weights': 15.0,
 'max_used_weights': 36.0,
 'median_r2_full': 0.5926642417907715,
 'min_r2_full': 0.5762367248535156,
 'max_r2_full': 0.5994216203689575,
 'median_r2_median': 0.6021612882614136,
 'min_r2_median': 0.5747120976448059,
 'max_r2_median': 0.6058254837989807,
 'median_corr_full': 0.7698462375702895,
 'min_corr_full': 0.7591025786107669,
 'max_corr_full': 0.7742232367792622,
 'median_corr_median': 0.7759905079284137,
 'min_corr_median': 0.7580976834450861,
 'max_corr_median': 0.7783479195057829}

In [6]:
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats

import os
import sys
current_dir = os.getcwd()
sys.path.append('layers')
from config import config

# Parameters from config file
HIDDEN_LAYERS = config['n_layers'] - 2 
epochs = config['num_epochs']
dim = config['hidden_dim']
n_nets = config['n_nets']

path = "results/abalone/flow/"
res = {}


res["used_weights"] = []

for n in range(n_nets):
    median_info = np.load(path+f"net{n}_median.npy", allow_pickle=True).item()
    res["used_weights"].append(median_info["used_weights"])

print("USED WEIGHTS:")
m = (res["used_weights"])
print(f"median: {np.median(m)}")
print(f"min: {np.min(m)}")
print(f"max: {np.max(m)}\n")

USED WEIGHTS:
median: 26.0
min: 15
max: 36



In [7]:
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats

import os
import sys
current_dir = os.getcwd()
sys.path.append('layers')
from config import config

# Parameters from config file
HIDDEN_LAYERS = config['n_layers'] - 2 
epochs = config['num_epochs']
dim = config['hidden_dim']
n_nets = config['n_nets']

path = "results/abalone/flow/"
res = {}
res["median_depth"] = []
res["max_width"] = []

for n in range(n_nets):
    median_info = np.load(path+f"net{n}_median.npy", allow_pickle=True).item()
    res["median_depth"].append(median_info["avg_path_length"])
    search = True
    i = 0
    while search:
        search = np.sum(median_info["include_inputs"][i]) == 0
        i += 1

    res["max_width"].append(HIDDEN_LAYERS-i+2)

print("Avg. depth")
print(np.median(res["median_depth"]))
print(np.min(res["median_depth"]))
print(np.max(res["median_depth"]))
print("")

print("Max depth")
print(np.median(res["max_width"]))
print(np.min(res["max_width"]))
print(np.max(res["max_width"]))
print("")

Avg. depth
1.8571428571428572
1.6363636363636365
1.9285714285714286

Max depth
2.0
2
3



In [26]:
import numpy as np


import os
import sys
current_dir = os.getcwd()
sys.path.append('layers')
from config import config

# Parameters from config file
HIDDEN_LAYERS = config['n_layers'] - 2 
epochs = config['num_epochs']
dim = config['hidden_dim']
n_nets = config['n_nets']

path = "results/abalone/flow/"
res = {}
for i in range(p):
    res[f"x{i+1}_inclution_rate"] = 0
    
    for n in range(n_nets):
        median_info = np.load(path+f"net{n}_median.npy", allow_pickle=True).item()
        #res[d][f"x{i+1}_inclution_rate"] += (median_info["expected_depth_input"][i]>0)
        count = 0
        for j in median_info["include_inputs"]:
            count += j[i]

        res[f"x{i+1}_inclution_rate"] += (count>0)
    res[f"x{i+1}_inclution_rate"] /= n_nets

res

{'x1_inclution_rate': 0.0,
 'x2_inclution_rate': 0.0,
 'x3_inclution_rate': 0.7,
 'x4_inclution_rate': 1.0,
 'x5_inclution_rate': 1.0,
 'x6_inclution_rate': 0.0,
 'x7_inclution_rate': 1.0,
 'x8_inclution_rate': 1.0,
 'x9_inclution_rate': 0.0}